In [2]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from translate import Translator
import re
from unicodedata import normalize
import string

# Lectura de los datos

In [4]:
path_train="C:/Users/Esperanza/Desktop/UOC/TFM/python/jigsaw-unintended-bias-in-toxicity-classification/train.csv"
df_train = pd.read_csv(path_train)

In [24]:
# Selección de las columnas de interes
df_train_samp=df_train[['target', 'comment_text']]

## Análisis vaderSentiment 

In [11]:
# CReamos el analizador de sentimientos de Vader
analyser = SentimentIntensityAnalyzer()    

En este módulo se comprueba algunas de las frases que fueron etiquetadas de forma incorrecta en Vader. Se quiere comprobar como hubieran sido evaluadas por el modelo de Vader

In [ ]:
#Analizamos cada uno de los comentarios añadiendo una columna con los resultados
df_train_samp['neg_vader']=df_train_samp.loc[:,'comment_text'].map(lambda txt: str(analyser.polarity_scores(txt)['neg']) )

In [ ]:
df_train_samp.loc[1:5,'neg_vader']

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(df_train_samp.target, df_train_samp.neg_vader)